In [1]:
import torch
import pyro
from pyro.optim import Adam, ClippedAdam
import congas as cg
from congas.models import LatentCategorical
from pyro.infer import TraceMeanField_ELBO,TraceEnum_ELBO, TraceGraph_ELBO



Bad key "text.kerning_factor" on line 4 in
/Users/salvatore/miniconda3/envs/hmobster/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import pickle

data_file = open("data_input_lucrezia.pkl",'rb')

data = pickle.loads(data_file.read())
data_file.close()

param_file =  open("pars_lucrezia.pkl",'rb')

param = pickle.loads(param_file.read())
param_file.close()


interface = cg.Interface()

In [3]:
pyro.enable_validation()



In [4]:
param["init_probs"] = 0.55
param["latent_type"] = "G"
param["lambda"] = 0.3
param["K"] = 1
param["Temperature"] = 1
param["equal_sizes_sd"] = True

In [5]:
interface.set_model(LatentCategorical)
interface.set_optimizer(ClippedAdam)
interface.set_loss(TraceGraph_ELBO)
interface.initialize_model(data)
interface.set_model_params(param)

In [6]:
ll = interface.run(steps = 80, param_optimizer = {"lr":0.1})



Running LatentCategorical on 3768 cells with 20 segments for 80 steps and 2 modalities
tensor([[3.6690, 3.4735, 3.7271, 3.3599, 3.4903, 3.3838, 3.6620, 3.4728, 3.5274,
         3.5362]])
tensor(-519094.7332, dtype=torch.float64)
tensor(18884.)
tensor(7881.2939)
tensor([[3.4035, 3.6292, 3.6570, 3.3821, 3.3159, 3.6078, 3.6281, 3.4522, 3.3757,
         3.8988]], grad_fn=<MulBackward0>)
tensor(-532933.9860, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(18884., grad_fn=<AddBackward0>)
tensor(7564.9019, grad_fn=<AddBackward0>)
tensor([[4.5043, 4.2099, 3.7352, 4.3224, 5.7310, 3.3941, 2.7637, 3.0060, 3.6185,
         3.9986]], grad_fn=<MulBackward0>)
tensor(-383284.0482, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(18884., grad_fn=<AddBackward0>)
tensor(8276.3594, grad_fn=<AddBackward0>)
tensor([[3.6955, 4.8268, 3.5709, 2.4900, 1.6660, 3.0001, 4.0572, 2.6396, 5.1010,
         2.1787]], grad_fn=<MulBackward0>)
tensor(-516548.9966, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor

ELBO: -295432.560509554  : 100%|██████████| 80/80 [00:03<00:00, 23.08it/s]


In [7]:

import numpy as np

lr = interface.learned_parameters()
ICs = interface.calculate_ICs()


Computing assignment probabilities
Computing information criteria.


In [8]:
lr

{'mixture_weights_rna': array([1.], dtype=float32),
 'mixture_weights_atac': array([1.], dtype=float32),
 'NB_size_rna': array([10.064728 , 10.050139 , 10.072672 , 10.061152 , 10.080033 ,
        10.060694 , 10.529044 , 10.053069 , 10.054522 , 10.0610285],
       dtype=float32),
 'segment_factor_rna': array([18.643011, 28.849041, 20.717102, 26.578852, 23.443935, 46.30036 ,
        12.231085, 22.868774, 32.068203, 21.373848], dtype=float32),
 'NB_size_atac': array([19.88664  , 10.23971  , 10.056137 , 10.20803  , 10.060608 ,
        10.070222 , 29.04216  , 10.072636 , 10.2402525, 10.085367 ],
       dtype=float32),
 'segment_factor_atac': array([13.385825, 36.953133, 20.348621, 35.795895, 31.493282, 55.537548,
        19.164227, 40.850975, 39.58086 , 44.045616], dtype=float32),
 'CNV_probabilities': array([[[2.85151327e-04, 2.36564534e-04, 2.31437836e-04, 4.88827471e-04,
          6.49931724e-04, 9.98108149e-01],
         [9.95771706e-01, 1.76130445e-03, 9.82055441e-04, 1.98913724e-04,
 

In [17]:

data["pld"]

tensor([2, 4, 5, 5, 4, 4, 2, 5, 5, 5])

In [43]:
from congas import log_sum_exp
lk_atac = lk_atac + torch.log(lr_t["mixture_weights_atac"]).reshape([2,1,1])

In [46]:
torch.exp(lk_atac - log_sum_exp(lk_atac))

tensor([[[9.9939e-01, 9.9989e-01, 1.4069e-04,  ..., 6.5166e-03,
          5.3351e-01, 1.5664e-07],
         [9.3591e-01, 1.6383e-01, 1.8467e-02,  ..., 6.9422e-01,
          8.2800e-01, 1.4531e-01],
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01],
         ...,
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01],
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01],
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01]],

        [[6.1418e-04, 1.1402e-04, 9.9986e-01,  ..., 9.9348e-01,
          4.6649e-01, 1.0000e+00],
         [6.4089e-02, 8.3617e-01, 9.8153e-01,  ..., 3.0578e-01,
          1.7200e-01, 8.5469e-01],
         [4.9646e-01, 4.9646e-01, 4.9646e-01,  ..., 4.9646e-01,
          4.9646e-01, 4.9646e-01],
         ...,
         [4.9646e-01, 4.9646e-01, 4.9646e-01,  ..., 4.9646e-01,
          4.964

In [7]:
data["data_atac"].shape

torch.Size([10, 1895])

In [10]:
data["pld"]

tensor([2, 4, 5, 5, 4, 4, 2, 5, 5, 5])